<a href="https://colab.research.google.com/github/Artursf/datasharing/blob/master/Learnings_%26_Conclusions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 - EXP 03 - Soft Bounce Reasons (Blacklists)



[Kickoff Document](https://docs.google.com/document/d/16fKgRXzWQeK8c4vDU2UAmARrKSHwpZ_2L3JA4lQybw8/edit)

```
01 Why are we doing this? What problem/opportunity is the main motivator for this experiment?
02 How the experiment impacts the problem/opportunity presented before?
```

[Hypothesis:](https://docs.google.com/document/d/1RirYIEHIeITRUQA-DhX6TFL7MLetMm95c9WABytoWJk/edit#heading=h.6jynaot9cbnq)

| By identifying and removing non-sorbs blacklisted IPs in a specific pool, we expect to **reduce** the soft-bounce events from the RDSM accounts by an average of 20% in this pool up to a week or two. |
|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|

[Learnings & Conclusions](https://docs.google.com/document/d/1wmAPhODKB3qcOFlsOpLqrDa6R1uHa0FJ4SYMhGjyW7A/edit#heading=h.6jynaot9cbnq)
______________________________________________________________________________

Code: [_Git_](https://github.com/ResultadosDigitais/data-science/tree/master/03%20Blacklists)

In [1]:
import pandas as pd
import statistics as stat
import numpy as np
import math as math

# 02 - Load EXP 03 - Data


In [2]:
#233310:
url_1 = 'https://raw.githubusercontent.com/ResultadosDigitais/data-science/master/03%20Blacklists/233310/233310_(Control_Group).csv?token=AEENAZWOGFWEFDRWPDWLPB267MYZM'
url_2 = 'https://raw.githubusercontent.com/ResultadosDigitais/data-science/master/03%20Blacklists/233310/233310_(Test_Group).csv?token=AEENAZVPXZVH4KMTIMSFTDC67MY2C'
old_shared_CONTROL = pd.read_csv(url_1)
old_shared_TEST = pd.read_csv(url_2)

#bad_excellent:
url_3 = 'https://raw.githubusercontent.com/ResultadosDigitais/data-science/master/03%20Blacklists/Bad_Excellent/Bad_Exc_(Control_Group).csv?token=AEENAZUNAXNKE77CBDDA2AK67MY6E'
url_4 = 'https://raw.githubusercontent.com/ResultadosDigitais/data-science/master/03%20Blacklists/Bad_Excellent/Bad_Exc_(Test_Group).csv?token=AEENAZTWI6T46PJ2EGEMRYC67MY6W'
BE_CONTROL = pd.read_csv(url_3)
BE_TEST = pd.read_csv(url_4)

#Magic Johnson:
url_5 = 'https://raw.githubusercontent.com/ResultadosDigitais/data-science/master/03%20Blacklists/Magic_Johnson/MJ_(Control_Group).csv?token=AEENAZXYK2NQL7GKNH4XIG267MZAG'
url_6 = 'https://raw.githubusercontent.com/ResultadosDigitais/data-science/master/03%20Blacklists/Magic_Johnson/MJ_(Test_Group).csv?token=AEENAZVXJVCAHEAXR6OZG2C67MZA6'
MJ_CONTROL = pd.read_csv(url_5)
MJ_TEST = pd.read_csv(url_6)

In [3]:
#Dataframe standard columns names:
old_shared_CONTROL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   day                          102 non-null    object 
 1   account_count                102 non-null    int64  
 2   campaign_count               102 non-null    int64  
 3   campaign_blocked_count       102 non-null    int64  
 4   rdsm_campaign_leads_count    102 non-null    int64  
 5   datalake_email_events_count  102 non-null    int64  
 6   processed_count              102 non-null    int64  
 7   delivered_count              102 non-null    int64  
 8   opened_count                 102 non-null    int64  
 9   clicked_count                102 non-null    int64  
 10  soft_bounced_count           102 non-null    int64  
 11  opened_count_unique          102 non-null    int64  
 12  clicked_count_unique         102 non-null    int64  
 13  delivered_rate      

In [4]:
#Change "day" schema to date:

#233310:
old_shared_CONTROL['day'] = pd.to_datetime(old_shared_CONTROL['day'])
old_shared_CONTROL.set_index('day', inplace=False)

old_shared_TEST['day'] = pd.to_datetime(old_shared_TEST['day'])
old_shared_TEST.set_index('day', inplace=False)

#bad_excellent:
BE_CONTROL['day'] = pd.to_datetime(BE_CONTROL['day'])
BE_CONTROL.set_index('day', inplace=False)

BE_TEST['day'] = pd.to_datetime(BE_TEST['day'])
BE_TEST.set_index('day', inplace=False)

#Magic Johnson:
MJ_CONTROL['day'] = pd.to_datetime(MJ_CONTROL['day'])
MJ_CONTROL.set_index('day', inplace=False)

MJ_TEST['day'] = pd.to_datetime(MJ_TEST['day'])
MJ_TEST.set_index('day', inplace=False)

,account_count,campaign_count,campaign_blocked_count,rdsm_campaign_leads_count,datalake_email_events_count,processed_count,delivered_count,opened_count,clicked_count,soft_bounced_count,opened_count_unique,clicked_count_unique,delivered_rate,opened_rate,clicked_rate,bounce_rate,soft_bounce_rate
day,,,,,,,,,,,,,,,,,
2020-06-10,6,9,1,95027,226025,88455,84971,14391,2812,2565,8320,1220,0.960613,0.097916,0.014358,0.050297,0.028986
2020-06-09,6,16,0,103299,240910,103253,102386,21544,1729,746,11183,1263,0.991603,0.109226,0.012336,0.009424,0.007167
2020-06-08,3,3,0,40000,89511,39990,39818,7289,717,203,3871,550,0.995699,0.097217,0.013813,0.005501,0.005076
2020-06-07,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2020-06-06,1,1,0,627,1863,627,626,422,164,1,186,88,0.998405,0.297125,0.140575,0.001595,0.001595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-05,2,5,1,9292,16784,6743,6031,1198,162,150,529,75,0.894674,0.087713,0.012436,0.118528,0.022252
2020-03-04,3,8,1,40223,85421,37743,35592,5671,863,1918,2982,323,0.943009,0.083783,0.009075,0.078902,0.050764
2020-03-03,2,5,1,11726,33776,9712,8441,4278,535,1138,1790,243,0.869117,0.212085,0.028791,0.163938,0.117187


In [5]:
#Dataframe standard columns data:
old_shared_CONTROL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   day                          102 non-null    datetime64[ns]
 1   account_count                102 non-null    int64         
 2   campaign_count               102 non-null    int64         
 3   campaign_blocked_count       102 non-null    int64         
 4   rdsm_campaign_leads_count    102 non-null    int64         
 5   datalake_email_events_count  102 non-null    int64         
 6   processed_count              102 non-null    int64         
 7   delivered_count              102 non-null    int64         
 8   opened_count                 102 non-null    int64         
 9   clicked_count                102 non-null    int64         
 10  soft_bounced_count           102 non-null    int64         
 11  opened_count_unique          102 non-null    

# ----------  Paired Samples (Dependent)

# 03 - Base Metrics Calculation - Paired Samples (Dependent)

In [7]:
## BEFORE (From '2020-05-11' to '2020-05-26') ~ 15 days
#SB data before experiment:

start_date_1 = '2020-05-11'
end_date_1 = '2020-05-25'

#233310:
mask_data_before_exp_old_shared_CONTROL = (old_shared_CONTROL['day'] > start_date_1) & (old_shared_CONTROL['day'] <= end_date_1)
data_before_exp_old_shared_CONTROL = old_shared_CONTROL.loc[mask_data_before_exp_old_shared_CONTROL]
order_data_before_exp_old_shared_CONTROL = data_before_exp_old_shared_CONTROL.reset_index(drop=True)

mask_data_before_exp_old_shared_TEST = (old_shared_TEST['day'] > start_date_1) & (old_shared_TEST['day'] <= end_date_1)
data_before_exp_old_shared_TEST = old_shared_TEST.loc[mask_data_before_exp_old_shared_TEST]
order_data_before_exp_old_shared_TEST = data_before_exp_old_shared_TEST.reset_index(drop=True)

#bad_excellent:
mask_data_before_exp_BE_CONTROL = (BE_CONTROL['day'] > start_date_1) & (BE_CONTROL['day'] <= end_date_1)
data_before_exp_BE_CONTROL = BE_CONTROL.loc[mask_data_before_exp_BE_CONTROL]
order_data_before_exp_BE_CONTROL = data_before_exp_BE_CONTROL.reset_index(drop=True)

mask_data_before_exp_BE_TEST = (BE_TEST['day'] > start_date_1) & (BE_TEST['day'] <= end_date_1)
data_before_exp_BE_TEST = BE_TEST.loc[mask_data_before_exp_BE_TEST]
order_data_before_exp_BE_TEST = data_before_exp_BE_TEST.reset_index(drop=True)

#Magic Johnson:
mask_data_before_exp_MJ_CONTROL = (MJ_CONTROL['day'] > start_date_1) & (MJ_CONTROL['day'] <= end_date_1)
data_before_exp_MJ_CONTROL = MJ_CONTROL.loc[mask_data_before_exp_MJ_CONTROL]
order_data_before_exp_MJ_CONTROL = data_before_exp_MJ_CONTROL.reset_index(drop=True)

mask_data_before_exp_MJ_TEST = (MJ_TEST['day'] > start_date_1) & (MJ_TEST['day'] <= end_date_1)
data_before_exp_MJ_TEST = MJ_TEST.loc[mask_data_before_exp_MJ_TEST]
order_data_before_exp_MJ_TEST = data_before_exp_MJ_TEST.reset_index(drop=True)

In [8]:
## AFTER (From '2020-05-26' to '2020-06-11') ~ 15 days
#SB data after experiment:

start_date_2 = '2020-05-27'
end_date_2 = '2020-06-11'

#233310:
mask_data_after_exp_old_shared_CONTROL = (old_shared_CONTROL['day'] > start_date_2) & (old_shared_CONTROL['day'] <= end_date_2)
data_after_exp_old_shared_CONTROL = old_shared_CONTROL.loc[mask_data_after_exp_old_shared_CONTROL]
order_data_after_exp_old_shared_CONTROL = data_after_exp_old_shared_CONTROL.reset_index(drop=True)

mask_data_after_exp_old_shared_TEST = (old_shared_TEST['day'] > start_date_2) & (old_shared_TEST['day'] <= end_date_2)
data_after_exp_old_shared_TEST = old_shared_TEST.loc[mask_data_after_exp_old_shared_TEST]
order_data_after_exp_old_shared_TEST = data_after_exp_old_shared_TEST.reset_index(drop=True)

#bad_excellent:
mask_data_after_exp_BE_CONTROL = (BE_CONTROL['day'] > start_date_2) & (BE_CONTROL['day'] <= end_date_2)
data_after_exp_BE_CONTROL = BE_CONTROL.loc[mask_data_after_exp_BE_CONTROL]
order_data_after_exp_BE_CONTROL = data_after_exp_BE_CONTROL.reset_index(drop=True)

mask_data_after_exp_BE_TEST = (BE_TEST['day'] > start_date_2) & (BE_TEST['day'] <= end_date_2)
data_after_exp_BE_TEST = BE_TEST.loc[mask_data_after_exp_BE_TEST]
order_data_after_exp_BE_TEST = data_after_exp_BE_TEST.reset_index(drop=True)

#Magic Johnson:
mask_data_after_exp_MJ_CONTROL = (MJ_CONTROL['day'] > start_date_2) & (MJ_CONTROL['day'] <= end_date_2)
data_after_exp_MJ_CONTROL = MJ_CONTROL.loc[mask_data_after_exp_MJ_CONTROL]
order_data_after_exp_MJ_CONTROL = data_after_exp_MJ_CONTROL.reset_index(drop=True)

mask_data_after_exp_MJ_TEST = (MJ_TEST['day'] > start_date_2) & (MJ_TEST['day'] <= end_date_2)
data_after_exp_MJ_TEST = MJ_TEST.loc[mask_data_after_exp_MJ_TEST]
order_data_after_exp_MJ_TEST = data_after_exp_MJ_TEST.reset_index(drop=True)

In [9]:
## BEFORE (From '2020-05-11' to '2020-05-26') ~ 15 days
#SB data before experiment

## AFTER (From '2020-05-26' to '2020-06-11') ~ 15 days
#SB data after experiment

#233310:
order_data_before_exp_old_shared_CONTROL[np.isnan(order_data_before_exp_old_shared_CONTROL['soft_bounce_rate'])] = 0
order_data_after_exp_old_shared_CONTROL[np.isnan(order_data_after_exp_old_shared_CONTROL['soft_bounce_rate'])] = 0
order_data_before_exp_old_shared_TEST[np.isnan(order_data_before_exp_old_shared_TEST['soft_bounce_rate'])] = 0
order_data_after_exp_old_shared_TEST[np.isnan(order_data_after_exp_old_shared_TEST['soft_bounce_rate'])] = 0

#bad_excellent:
order_data_before_exp_BE_CONTROL[np.isnan(order_data_before_exp_BE_CONTROL['soft_bounce_rate'])] = 0
order_data_after_exp_BE_CONTROL[np.isnan(order_data_after_exp_BE_CONTROL['soft_bounce_rate'])] = 0
order_data_before_exp_BE_TEST[np.isnan(order_data_before_exp_BE_TEST['soft_bounce_rate'])] = 0
order_data_after_exp_BE_TEST[np.isnan(order_data_after_exp_BE_TEST['soft_bounce_rate'])] = 0

#Magic Johnson:
order_data_before_exp_MJ_CONTROL[np.isnan(order_data_before_exp_MJ_CONTROL['soft_bounce_rate'])] = 0
order_data_after_exp_MJ_CONTROL[np.isnan(order_data_after_exp_MJ_CONTROL['soft_bounce_rate'])] = 0
order_data_before_exp_MJ_TEST[np.isnan(order_data_before_exp_MJ_TEST['soft_bounce_rate'])] = 0
order_data_after_exp_MJ_TEST[np.isnan(order_data_after_exp_MJ_TEST['soft_bounce_rate'])] = 0

#__________________________________________________________________________

#233310:
data_diff_exp_old_shared_CONTROL = order_data_before_exp_old_shared_CONTROL['soft_bounce_rate'] - order_data_after_exp_old_shared_CONTROL['soft_bounce_rate']
data_diff_exp_old_shared_TEST = order_data_before_exp_old_shared_TEST['soft_bounce_rate'] - order_data_after_exp_old_shared_TEST['soft_bounce_rate']

#bad_excellent:
data_diff_exp_BE_CONTROL = order_data_before_exp_BE_CONTROL['soft_bounce_rate'] - order_data_after_exp_BE_CONTROL['soft_bounce_rate']
data_diff_exp_BE_TEST = order_data_before_exp_BE_TEST['soft_bounce_rate'] - order_data_after_exp_BE_TEST['soft_bounce_rate']

#Magic Johnson:
data_diff_exp_MJ_CONTROL = order_data_before_exp_MJ_CONTROL['soft_bounce_rate'] - order_data_after_exp_MJ_CONTROL['soft_bounce_rate']
data_diff_exp_MJ_TEST = order_data_before_exp_MJ_TEST['soft_bounce_rate'] - order_data_after_exp_MJ_TEST['soft_bounce_rate']

In [47]:
#SUM diff - Σ(D)

#233310:
SUM_data_diff_exp_old_shared_CONTROL = data_diff_exp_old_shared_CONTROL.sum()
SUM_data_diff_exp_old_shared_TEST = data_diff_exp_old_shared_TEST.sum()

#bad_excellent:
SUM_data_diff_exp_BE_CONTROL = data_diff_exp_BE_CONTROL.sum()
SUM_data_diff_exp_BE_TEST = data_diff_exp_BE_TEST.sum()

#Magic Johnson:
SUM_data_diff_exp_MJ_CONTROL = data_diff_exp_MJ_CONTROL.sum()
SUM_data_diff_exp_MJ_TEST = data_diff_exp_MJ_TEST.sum()

In [48]:
#SQUARE - (D ** 2)

#233310:
SQRT_data_diff_exp_old_shared_CONTROL = data_diff_exp_old_shared_CONTROL * data_diff_exp_old_shared_CONTROL
SQRT_data_diff_exp_old_shared_TEST = data_diff_exp_old_shared_TEST * data_diff_exp_old_shared_TEST

#bad_excellent:
SQRT_data_diff_exp_BE_CONTROL = data_diff_exp_BE_CONTROL * data_diff_exp_BE_CONTROL
SQRT_data_diff_exp_BE_TEST = data_diff_exp_BE_TEST * data_diff_exp_BE_TEST

#Magic Johnson:
SQRT_data_diff_exp_MJ_CONTROL = data_diff_exp_MJ_CONTROL * data_diff_exp_MJ_CONTROL
SQRT_data_diff_exp_MJ_TEST = data_diff_exp_MJ_TEST * data_diff_exp_MJ_TEST

In [49]:
#SUM SQUARE - Σ ( (D) ** 2 )

#233310:
SUM_SQRT_data_diff_exp_old_shared_CONTROL = SQRT_data_diff_exp_old_shared_CONTROL.sum()
SUM_SQRT_data_diff_exp_old_shared_TEST = SQRT_data_diff_exp_old_shared_TEST.sum()

#bad_excellent:
SUM_SQRT_data_diff_exp_BE_CONTROL = SQRT_data_diff_exp_BE_CONTROL.sum()
SUM_SQRT_data_diff_exp_BE_TEST = SQRT_data_diff_exp_BE_TEST.sum()

#Magic Johnson:
SUM_SQRT_data_diff_exp_MJ_CONTROL = SQRT_data_diff_exp_MJ_CONTROL.sum()
SUM_SQRT_data_diff_exp_MJ_TEST = SQRT_data_diff_exp_MJ_TEST.sum()

In [50]:
# Soft Bounce (SB) datapoints:

#233310:
diff_old_shared_CONTROL_SB_datapoints = data_diff_exp_old_shared_CONTROL.count()
diff_old_shared_TEST_SB_datapoints = data_diff_exp_old_shared_TEST.count()

#bad_excellent:
diff_BE_CONTROL_SB_datapoints = data_diff_exp_BE_CONTROL.count()
diff_BE_TEST_SB_datapoints = data_diff_exp_BE_TEST.count()

#Magic Johnson:
diff_MJ_CONTROL_SB_datapoints = data_diff_exp_MJ_CONTROL.count()
diff_MJ_TEST_SB_datapoints = data_diff_exp_MJ_TEST.count()

# 04 - Degrees of Freedom & Critical-Value & T-Table - Paired Samples (Dependent)


In [51]:
## BEFORE (From '2020-05-11' to '2020-05-26') ~ 15 days
## AFTER (From '2020-05-26' to '2020-06-11') ~ 15 days

# Soft Bounce (SB) Degrees of Freedom: - (N)

#233310:
df_diff_exp_old_shared_CONTROL = diff_old_shared_CONTROL_SB_datapoints - 1
df_diff_exp_old_shared_TEST    = diff_old_shared_TEST_SB_datapoints - 1

#bad_excellent:
df_diff_exp_BE_CONTROL = diff_BE_CONTROL_SB_datapoints - 1
df_diff_exp_BE_TEST    = diff_BE_TEST_SB_datapoints - 1

#Magic Johnson:
df_diff_exp_MJ_CONTROL = diff_MJ_CONTROL_SB_datapoints - 1
df_diff_exp_MJ_TEST    = diff_MJ_TEST_SB_datapoints - 1

In [52]:
print('Degrees of Freedom:')
print(' ')
print('#233310:')
print('df_diff_exp_old_shared_CONTROL = {}'.format(df_diff_exp_old_shared_CONTROL))
print('df_diff_exp_old_shared_TEST = {}'.format(df_diff_exp_old_shared_TEST))
print(' ')
print('#bad_excellent:')
print('df_diff_exp_BE_CONTROL = {}'.format(df_diff_exp_BE_CONTROL))
print('df_diff_exp_BE_TEST = {}'.format(df_diff_exp_BE_TEST))
print(' ')
print('#Magic Johnson:')
print('df_diff_exp_MJ_CONTROL = {}'.format(df_diff_exp_MJ_CONTROL))
print('df_diff_exp_MJ_TEST = {}'.format(df_diff_exp_MJ_TEST))

Degrees of Freedom:
 
#233310:
df_diff_exp_old_shared_CONTROL = 13
df_diff_exp_old_shared_TEST = 13
 
#bad_excellent:
df_diff_exp_BE_CONTROL = 13
df_diff_exp_BE_TEST = 13
 
#Magic Johnson:
df_diff_exp_MJ_CONTROL = 13
df_diff_exp_MJ_TEST = 13


![t-table-chosen](https://lh6.googleusercontent.com/-4YHhiUxkXTrksyXJqKzedw_pq55RDvGwVNS7srZx5Ts4tmV1iTAkC0a3hdBz6KO_sD7IK09_Njr2yCegrgz=w1920-h1007)

In [58]:
# From the t-table:
#_______________________________________________________
# Alpha-value ("Significance Level") of 5%:   (Balanced Type I and Type II error probability)

critical_value_diff_exp = 2.160  #13

# 05 - t-value - Paired Samples (Dependent)

Paired Samples (Dependent)

http://www.statstutor.ac.uk/resources/uploaded/paired-t-test.pdf

![texto alternativo](https://lh3.googleusercontent.com/lnzSnpmkCL09H6cn0Y_Eukn_BnqaZf0CSy2dy7RDCw2HrHzSQgflKSU4lu9o5M-EiI8MnZauPHR6UUa7Q7to=w1920-h1007)


In [56]:
# Unpaired Samples (Independent):

#233310:
t_value_paired_exp_old_shared_CONTROL = ( SUM_data_diff_exp_old_shared_CONTROL / df_diff_exp_old_shared_CONTROL ) / ( math.sqrt( ( (SUM_SQRT_data_diff_exp_old_shared_CONTROL) - ( ((SUM_data_diff_exp_old_shared_CONTROL) ** 2) / df_diff_exp_old_shared_CONTROL )) / ( (df_diff_exp_old_shared_CONTROL - 1) * df_diff_exp_old_shared_CONTROL)))
t_value_paired_exp_old_shared_TEST = ( SUM_data_diff_exp_old_shared_TEST / df_diff_exp_old_shared_TEST ) / ( math.sqrt( ( (SUM_SQRT_data_diff_exp_old_shared_TEST) - ( ((SUM_data_diff_exp_old_shared_TEST) ** 2) / df_diff_exp_old_shared_TEST )) / ( (df_diff_exp_old_shared_TEST - 1) * df_diff_exp_old_shared_TEST)))

#bad_excellent:
t_value_paired_exp_BE_CONTROL = ( SUM_data_diff_exp_BE_CONTROL / df_diff_exp_BE_CONTROL ) / ( math.sqrt( ( (SUM_SQRT_data_diff_exp_BE_CONTROL) - ( ((SUM_data_diff_exp_BE_CONTROL) ** 2) / df_diff_exp_BE_CONTROL )) / ( (df_diff_exp_BE_CONTROL - 1) * df_diff_exp_BE_CONTROL)))
t_value_paired_exp_BE_TEST = ( SUM_data_diff_exp_BE_TEST / df_diff_exp_BE_TEST ) / ( math.sqrt( ( (SUM_SQRT_data_diff_exp_BE_TEST) - ( ((SUM_data_diff_exp_BE_TEST) ** 2) / df_diff_exp_BE_TEST )) / ( (df_diff_exp_BE_TEST - 1) * df_diff_exp_BE_TEST)))

#Magic Johnson:
t_value_paired_exp_MJ_CONTROL = ( SUM_data_diff_exp_MJ_CONTROL / df_diff_exp_MJ_CONTROL ) / ( math.sqrt( ( (SUM_SQRT_data_diff_exp_MJ_CONTROL) - ( ((SUM_data_diff_exp_MJ_CONTROL) ** 2) / df_diff_exp_MJ_CONTROL )) / ( (df_diff_exp_MJ_CONTROL - 1) * df_diff_exp_MJ_CONTROL)))
t_value_paired_exp_MJ_TEST = ( SUM_data_diff_exp_MJ_TEST / df_diff_exp_MJ_TEST ) / ( math.sqrt( ( (SUM_SQRT_data_diff_exp_MJ_TEST) - ( ((SUM_data_diff_exp_MJ_TEST) ** 2) / df_diff_exp_MJ_TEST )) / ( (df_diff_exp_MJ_TEST - 1) * df_diff_exp_MJ_TEST)))

In [57]:
print('T-Values:')
print(' ')
print('#233310:')
print('t_value_paired_exp_old_shared_CONTROL = {}'.format(t_value_paired_exp_old_shared_CONTROL))
print('t_value_paired_exp_old_shared_TEST = {}'.format(t_value_paired_exp_old_shared_TEST))
print(' ')
print('#bad_excellent:')
print('t_value_paired_exp_BE_CONTROL = {}'.format(t_value_paired_exp_BE_CONTROL))
print('t_value_paired_exp_BE_TEST = {}'.format(t_value_paired_exp_BE_TEST))
print(' ')
print('#Magic Johnson:')
print('t_value_paired_exp_MJ_CONTROL = {}'.format(t_value_paired_exp_MJ_CONTROL))
print('t_value_paired_exp_MJ_TEST = {}'.format(t_value_paired_exp_MJ_TEST))

T-Values:
 
#233310:
t_value_paired_exp_old_shared_CONTROL = 0.42203095039255506
t_value_paired_exp_old_shared_TEST = 0.19446562174654317
 
#bad_excellent:
t_value_paired_exp_BE_CONTROL = -1.9373691824564752
t_value_paired_exp_BE_TEST = -1.411741285143528
 
#Magic Johnson:
t_value_paired_exp_MJ_CONTROL = 0.1780527378131308
t_value_paired_exp_MJ_TEST = 0.905201942481129


# 06 - T-Test Check - Paired Samples (Dependent)

In [60]:
# Verifying t-test:
# If T_Value > Critical_Value, then P_Value > .05

#233310:
print('#233310 - CONTROL:')
print(' ')
if t_value_paired_exp_old_shared_CONTROL > critical_value_diff_exp:
    print("REJECT the H0 -> There is statistically significant difference between the samples")
else:
    print("Don't Reject the H0 -> There is NO statistically significant difference between the samples")
print('t_value_exp_old_shared_CONTROL = {}'.format(t_value_paired_exp_old_shared_CONTROL))
print('critical_value_exp_old_shared_CONTROL = {}'.format(critical_value_diff_exp))
print(' ')
print('#233310 - TEST:')
print(' ')
if t_value_paired_exp_old_shared_TEST > critical_value_diff_exp:
    print("REJECT the H0 -> There is statistically significant difference between the samples")
else:
    print("Don't Reject the H0 -> There is NO statistically significant difference between the samples")
print('t_value_exp_old_shared_TEST = {}'.format(t_value_paired_exp_old_shared_TEST))
print('critical_value_exp_old_shared_TEST = {}'.format(critical_value_diff_exp))
print('________________________________________________________________________________________________________')
print(' ')

#______________________________________________________________________________________________________________
#bad_excellent:
print('#bad_excellent - CONTROL:')
print(' ')
if t_value_paired_exp_BE_CONTROL > critical_value_diff_exp:
    print("REJECT the H0 -> There is statistically significant difference between the samples")
else:
    print("Don't Reject the H0 -> There is NO statistically significant difference between the samples")
print('t_value_exp_BE_CONTROL = {}'.format(t_value_paired_exp_BE_CONTROL))
print('critical_value_exp_BE_CONTROL = {}'.format(critical_value_diff_exp))
print(' ')
print('#bad_excellent - TEST:')
print(' ')
if t_value_paired_exp_BE_TEST > critical_value_diff_exp:
    print("REJECT the H0 -> There is statistically significant difference between the samples")
else:
    print("Don't Reject the H0 -> There is NO statistically significant difference between the samples")
print('t_value_exp_BE_TEST = {}'.format(t_value_paired_exp_BE_TEST))
print('critical_value_exp_BE_TEST = {}'.format(critical_value_diff_exp))
print('________________________________________________________________________________________________________')
print(' ')

#______________________________________________________________________________________________________________
#Magic Johnson:
print('#Magic Johnson - CONTROL:')
print(' ')
if t_value_paired_exp_MJ_CONTROL > critical_value_diff_exp:
    print("REJECT the H0 -> There is statistically significant difference between the samples")
else:
    print("Don't Reject the H0 -> There is NO statistically significant difference between the samples")
print('t_value_exp_MJ_CONTROL = {}'.format(t_value_paired_exp_MJ_CONTROL))
print('critical_value_exp_MJ_CONTROL = {}'.format(critical_value_diff_exp))
print(' ')
print(' ')
print('#Magic Johnson - TEST:')
print(' ')
if t_value_paired_exp_MJ_TEST > critical_value_diff_exp:
    print("REJECT the H0 -> There is statistically significant difference between the samples")
else:
    print("Don't Reject the H0 -> There is NO statistically significant difference between the samples")
print('t_value_exp_MJ_TEST = {}'.format(t_value_paired_exp_MJ_TEST))
print('critical_value_exp_MJ_TEST = {}'.format(critical_value_diff_exp))

#233310 - CONTROL:
 
Don't Reject the H0 -> There is NO statistically significant difference between the samples
t_value_exp_old_shared_CONTROL = 0.42203095039255506
critical_value_exp_old_shared_CONTROL = 2.16
 
#233310 - TEST:
 
Don't Reject the H0 -> There is NO statistically significant difference between the samples
t_value_exp_old_shared_TEST = 0.19446562174654317
critical_value_exp_old_shared_TEST = 2.16
________________________________________________________________________________________________________
 
#bad_excellent - CONTROL:
 
Don't Reject the H0 -> There is NO statistically significant difference between the samples
t_value_exp_BE_CONTROL = -1.9373691824564752
critical_value_exp_BE_CONTROL = 2.16
 
#bad_excellent - TEST:
 
Don't Reject the H0 -> There is NO statistically significant difference between the samples
t_value_exp_BE_TEST = -1.411741285143528
critical_value_exp_BE_TEST = 2.16
__________________________________________________________________________________

# ----------  Unpaired Samples (Independent)




# Base Metrics Calculation - Unpaired Samples (Independent)



In [6]:
## ALL_TIME (From '2020-03-01' to '2020-06-11')
# Soft Bounce (SB) means:

#233310:
old_shared_CONTROL_Alltime_SB_mean = old_shared_CONTROL["soft_bounce_rate"].mean(skipna = True)
old_shared_TEST_Alltime_SB_mean = old_shared_TEST["soft_bounce_rate"].mean(skipna = True)

#bad_excellent:
BE_CONTROL_Alltime_SB_mean = BE_CONTROL["soft_bounce_rate"].mean(skipna = True)
BE_TEST_Alltime_SB_mean = BE_TEST["soft_bounce_rate"].mean(skipna = True)

#Magic Johnson:
MJ_CONTROL_Alltime_SB_mean = MJ_CONTROL["soft_bounce_rate"].mean(skipna = True)
MJ_TEST_Alltime_SB_mean = MJ_TEST["soft_bounce_rate"].mean(skipna = True)

In [7]:
print('All time SB means:')
print(' ')
print('#233310:')
print('old_shared_CONTROL_Alltime_SB_mean = {}'.format(old_shared_CONTROL_Alltime_SB_mean))
print('old_shared_TEST_Alltime_SB_mean = {}'.format(old_shared_TEST_Alltime_SB_mean))
print(' ')
print('#bad_excellent:')
print('BE_CONTROL_Alltime_SB_mean = {}'.format(BE_CONTROL_Alltime_SB_mean))
print('BE_TEST_Alltime_SB_mean = {}'.format(BE_TEST_Alltime_SB_mean))
print(' ')
print('#Magic Johnson:')
print('MJ_CONTROL_Alltime_SB_mean = {}'.format(MJ_CONTROL_Alltime_SB_mean))
print('MJ_TEST_Alltime_SB_mean = {}'.format(MJ_TEST_Alltime_SB_mean))

All time SB means:
 
#233310:
old_shared_CONTROL_Alltime_SB_mean = 0.019870591038961043
old_shared_TEST_Alltime_SB_mean = 0.023886321194029853
 
#bad_excellent:
BE_CONTROL_Alltime_SB_mean = 0.05221895317073171
BE_TEST_Alltime_SB_mean = 0.06862855013157895
 
#Magic Johnson:
MJ_CONTROL_Alltime_SB_mean = 0.03158165151162791
MJ_TEST_Alltime_SB_mean = 0.044390002972972964


## BEFORE EXPERIMENT ( < 2020-05-26)

In [19]:
## BEFORE (From '2020-05-11' to '2020-05-26') ~ 15 days
#SB data before experiment:

start_date_1 = '2020-05-11'
end_date_1 = '2020-05-25'

#233310:
mask_data_before_exp_old_shared_CONTROL = (old_shared_CONTROL['day'] > start_date_1) & (old_shared_CONTROL['day'] <= end_date_1)
data_before_exp_old_shared_CONTROL = old_shared_CONTROL.loc[mask_data_before_exp_old_shared_CONTROL]
order_data_before_exp_old_shared_CONTROL = data_before_exp_old_shared_CONTROL.reset_index(drop=True)

mask_data_before_exp_old_shared_TEST = (old_shared_TEST['day'] > start_date_1) & (old_shared_TEST['day'] <= end_date_1)
data_before_exp_old_shared_TEST = old_shared_TEST.loc[mask_data_before_exp_old_shared_TEST]
order_data_before_exp_old_shared_TEST = data_before_exp_old_shared_TEST.reset_index(drop=True)

#bad_excellent:
mask_data_before_exp_BE_CONTROL = (BE_CONTROL['day'] > start_date_1) & (BE_CONTROL['day'] <= end_date_1)
data_before_exp_BE_CONTROL = BE_CONTROL.loc[mask_data_before_exp_BE_CONTROL]
order_data_before_exp_BE_CONTROL = data_before_exp_BE_CONTROL.reset_index(drop=True)

mask_data_before_exp_BE_TEST = (BE_TEST['day'] > start_date_1) & (BE_TEST['day'] <= end_date_1)
data_before_exp_BE_TEST = BE_TEST.loc[mask_data_before_exp_BE_TEST]
order_data_before_exp_BE_TEST = data_before_exp_BE_TEST.reset_index(drop=True)

#Magic Johnson:
mask_data_before_exp_MJ_CONTROL = (MJ_CONTROL['day'] > start_date_1) & (MJ_CONTROL['day'] <= end_date_1)
data_before_exp_MJ_CONTROL = MJ_CONTROL.loc[mask_data_before_exp_MJ_CONTROL]
order_data_before_exp_MJ_CONTROL = data_before_exp_MJ_CONTROL.reset_index(drop=True)

mask_data_before_exp_MJ_TEST = (MJ_TEST['day'] > start_date_1) & (MJ_TEST['day'] <= end_date_1)
data_before_exp_MJ_TEST = MJ_TEST.loc[mask_data_before_exp_MJ_TEST]
order_data_before_exp_MJ_TEST = data_before_exp_MJ_TEST.reset_index(drop=True)

In [9]:
## BEFORE (From '2020-05-11' to '2020-05-26')
# Soft Bounce (SB) means:

#233310:
old_shared_CONTROL_before_SB_mean = data_before_exp_old_shared_CONTROL["soft_bounce_rate"].mean(skipna = True)
old_shared_TEST_before_SB_mean = data_before_exp_old_shared_TEST["soft_bounce_rate"].mean(skipna = True)

#bad_excellent:
BE_CONTROL_before_SB_mean = data_before_exp_BE_CONTROL["soft_bounce_rate"].mean(skipna = True)
BE_TEST_before_SB_mean = data_before_exp_BE_TEST["soft_bounce_rate"].mean(skipna = True)

#Magic Johnson:
MJ_CONTROL_before_SB_mean = data_before_exp_MJ_CONTROL["soft_bounce_rate"].mean(skipna = True)
MJ_TEST_before_SB_mean = data_before_exp_MJ_TEST["soft_bounce_rate"].mean(skipna = True)

In [10]:
## BEFORE (From '2020-05-11' to '2020-05-26')
# Soft Bounce (SB) standard deviation (s):

#233310:
old_shared_CONTROL_before_SB_stdev = np.nanstd(data_before_exp_old_shared_CONTROL["soft_bounce_rate"])
old_shared_TEST_before_SB_stdev = np.nanstd(data_before_exp_old_shared_TEST["soft_bounce_rate"])

#bad_excellent:
BE_CONTROL_before_SB_stdev = np.nanstd(data_before_exp_BE_CONTROL["soft_bounce_rate"])
BE_TEST_before_SB_stdev = np.nanstd(data_before_exp_BE_TEST["soft_bounce_rate"])

#Magic Johnson:
MJ_CONTROL_before_SB_stdev = np.nanstd(data_before_exp_MJ_CONTROL["soft_bounce_rate"])
MJ_TEST_before_SB_stdev = np.nanstd(data_before_exp_MJ_TEST["soft_bounce_rate"])

In [11]:
## BEFORE (From '2020-05-11' to '2020-05-26')
# Soft Bounce (SB) variance (s^2):

#233310:
old_shared_CONTROL_before_SB_variance = np.nanstd(data_before_exp_old_shared_CONTROL["soft_bounce_rate"])*np.nanstd(data_before_exp_old_shared_CONTROL["soft_bounce_rate"])
old_shared_TEST_before_SB_variance = np.nanstd(data_before_exp_old_shared_TEST["soft_bounce_rate"])*np.nanstd(data_before_exp_old_shared_TEST["soft_bounce_rate"])

#bad_excellent:
BE_CONTROL_before_SB_variance = np.nanstd(data_before_exp_BE_CONTROL["soft_bounce_rate"])*np.nanstd(data_before_exp_BE_CONTROL["soft_bounce_rate"])
BE_TEST_before_SB_variance = np.nanstd(data_before_exp_BE_TEST["soft_bounce_rate"])*np.nanstd(data_before_exp_BE_TEST["soft_bounce_rate"])

#Magic Johnson:
MJ_CONTROL_before_SB_variance = np.nanstd(data_before_exp_MJ_CONTROL["soft_bounce_rate"])*np.nanstd(data_before_exp_MJ_CONTROL["soft_bounce_rate"])
MJ_TEST_before_SB_variance = np.nanstd(data_before_exp_MJ_TEST["soft_bounce_rate"])*np.nanstd(data_before_exp_MJ_TEST["soft_bounce_rate"])

In [12]:
## BEFORE (From '2020-05-11' to '2020-05-26')
# Non-NaN Soft Bounce (SB) datapoints:

#233310:
old_shared_CONTROL_before_SB_datapoints = data_before_exp_old_shared_CONTROL['soft_bounce_rate'].count()
old_shared_TEST_before_SB_datapoints = data_before_exp_old_shared_TEST['soft_bounce_rate'].count()

#bad_excellent:
BE_CONTROL_before_SB_datapoints = data_before_exp_BE_CONTROL['soft_bounce_rate'].count()
BE_TEST_before_SB_datapoints = data_before_exp_BE_TEST['soft_bounce_rate'].count()

#Magic Johnson:
MJ_CONTROL_before_SB_datapoints = data_before_exp_MJ_CONTROL['soft_bounce_rate'].count()
MJ_TEST_before_SB_datapoints = data_before_exp_MJ_TEST['soft_bounce_rate'].count()

## AFTER EXPERIMENT ( > 2020-05-26)

In [20]:
## AFTER (From '2020-05-26' to '2020-06-11') ~ 15 days
#SB data after experiment:

start_date_2 = '2020-05-27'
end_date_2 = '2020-06-11'

#233310:
mask_data_after_exp_old_shared_CONTROL = (old_shared_CONTROL['day'] > start_date_2) & (old_shared_CONTROL['day'] <= end_date_2)
data_after_exp_old_shared_CONTROL = old_shared_CONTROL.loc[mask_data_after_exp_old_shared_CONTROL]
order_data_after_exp_old_shared_CONTROL = data_after_exp_old_shared_CONTROL.reset_index(drop=True)

mask_data_after_exp_old_shared_TEST = (old_shared_TEST['day'] > start_date_2) & (old_shared_TEST['day'] <= end_date_2)
data_after_exp_old_shared_TEST = old_shared_TEST.loc[mask_data_after_exp_old_shared_TEST]
order_data_after_exp_old_shared_TEST = data_after_exp_old_shared_TEST.reset_index(drop=True)

#bad_excellent:
mask_data_after_exp_BE_CONTROL = (BE_CONTROL['day'] > start_date_2) & (BE_CONTROL['day'] <= end_date_2)
data_after_exp_BE_CONTROL = BE_CONTROL.loc[mask_data_after_exp_BE_CONTROL]
order_data_after_exp_BE_CONTROL = data_after_exp_BE_CONTROL.reset_index(drop=True)

mask_data_after_exp_BE_TEST = (BE_TEST['day'] > start_date_2) & (BE_TEST['day'] <= end_date_2)
data_after_exp_BE_TEST = BE_TEST.loc[mask_data_after_exp_BE_TEST]
order_data_after_exp_BE_TEST = data_after_exp_BE_TEST.reset_index(drop=True)

#Magic Johnson:
mask_data_after_exp_MJ_CONTROL = (MJ_CONTROL['day'] > start_date_2) & (MJ_CONTROL['day'] <= end_date_2)
data_after_exp_MJ_CONTROL = MJ_CONTROL.loc[mask_data_after_exp_MJ_CONTROL]
order_data_after_exp_MJ_CONTROL = data_after_exp_MJ_CONTROL.reset_index(drop=True)

mask_data_after_exp_MJ_TEST = (MJ_TEST['day'] > start_date_2) & (MJ_TEST['day'] <= end_date_2)
data_after_exp_MJ_TEST = MJ_TEST.loc[mask_data_after_exp_MJ_TEST]
order_data_after_exp_MJ_TEST = data_after_exp_MJ_TEST.reset_index(drop=True)

In [14]:
## AFTER (From '2020-05-26' to '2020-06-11')
# Soft Bounce (SB) means:

#233310:
old_shared_CONTROL_after_SB_mean = data_after_exp_old_shared_CONTROL["soft_bounce_rate"].mean(skipna = True)
old_shared_TEST_after_SB_mean = data_after_exp_old_shared_TEST["soft_bounce_rate"].mean(skipna = True)

#bad_excellent:
BE_CONTROL_after_SB_mean = data_after_exp_BE_CONTROL["soft_bounce_rate"].mean(skipna = True)
BE_TEST_after_SB_mean = data_after_exp_BE_TEST["soft_bounce_rate"].mean(skipna = True)

#Magic Johnson:
MJ_CONTROL_after_SB_mean = data_after_exp_MJ_CONTROL["soft_bounce_rate"].mean(skipna = True)
MJ_TEST_after_SB_mean = data_after_exp_MJ_TEST["soft_bounce_rate"].mean(skipna = True)

In [15]:
## AFTER (From '2020-05-26' to '2020-06-11')
# Soft Bounce (SB) standard deviation (s):

#233310:
old_shared_CONTROL_after_SB_stdev = np.nanstd(data_after_exp_old_shared_CONTROL["soft_bounce_rate"])
old_shared_TEST_after_SB_stdev = np.nanstd(data_after_exp_old_shared_TEST["soft_bounce_rate"])

#bad_excellent:
BE_CONTROL_after_SB_stdev = np.nanstd(data_after_exp_BE_CONTROL["soft_bounce_rate"])
BE_TEST_after_SB_stdev = np.nanstd(data_after_exp_BE_TEST["soft_bounce_rate"])

#Magic Johnson:
MJ_CONTROL_after_SB_stdev = np.nanstd(data_after_exp_MJ_CONTROL["soft_bounce_rate"])
MJ_TEST_after_SB_stdev = np.nanstd(data_after_exp_MJ_TEST["soft_bounce_rate"])

In [16]:
## AFTER (From '2020-05-26' to '2020-06-11')
# Soft Bounce (SB) variance (s^2):

#233310:
old_shared_CONTROL_after_SB_variance = np.nanstd(data_after_exp_old_shared_CONTROL["soft_bounce_rate"])*np.nanstd(data_after_exp_old_shared_CONTROL["soft_bounce_rate"])
old_shared_TEST_after_SB_variance = np.nanstd(data_after_exp_old_shared_TEST["soft_bounce_rate"])*np.nanstd(data_after_exp_old_shared_TEST["soft_bounce_rate"])

#bad_excellent:
BE_CONTROL_after_SB_variance = np.nanstd(data_after_exp_BE_CONTROL["soft_bounce_rate"])*np.nanstd(data_after_exp_BE_CONTROL["soft_bounce_rate"])
BE_TEST_after_SB_variance = np.nanstd(data_after_exp_BE_TEST["soft_bounce_rate"])*np.nanstd(data_after_exp_BE_TEST["soft_bounce_rate"])

#Magic Johnson:
MJ_CONTROL_after_SB_variance = np.nanstd(data_after_exp_MJ_CONTROL["soft_bounce_rate"])*np.nanstd(data_after_exp_MJ_CONTROL["soft_bounce_rate"])
MJ_TEST_after_SB_variance = np.nanstd(data_after_exp_MJ_TEST["soft_bounce_rate"])*np.nanstd(data_after_exp_MJ_TEST["soft_bounce_rate"])

In [17]:
## AFTER (From '2020-05-26' to '2020-06-11')
# Non-NaN Soft Bounce (SB) datapoints:

#233310:
old_shared_CONTROL_after_SB_datapoints = data_after_exp_old_shared_CONTROL['soft_bounce_rate'].count()
old_shared_TEST_after_SB_datapoints = data_after_exp_old_shared_TEST['soft_bounce_rate'].count()

#bad_excellent:
BE_CONTROL_after_SB_datapoints = data_after_exp_BE_CONTROL['soft_bounce_rate'].count()
BE_TEST_after_SB_datapoints = data_after_exp_BE_TEST['soft_bounce_rate'].count()

#Magic Johnson:
MJ_CONTROL_after_SB_datapoints = data_after_exp_MJ_CONTROL['soft_bounce_rate'].count()
MJ_TEST_after_SB_datapoints = data_after_exp_MJ_TEST['soft_bounce_rate'].count()

# Degrees of Freedom & Critical-Value & T-Table - Unpaired Samples (Independent)



![t-test](https://lh5.googleusercontent.com/B_QMr6Ih3kgMA-QzkDSFLjeL2pyqwhYs2fhVR2FME9Cxq5UQxJz_8Omf4nl5tA3w1wZ0Pq3w6lKTbZ_GmJnZ=w1920-h1007-rw)

![p_value_table](https://www.biologyforlife.com/uploads/2/2/3/9/22392738/ttable.png)


In [ ]:
## BEFORE (From '2020-05-11' to '2020-05-26') ~ 15 days
## AFTER (From '2020-05-26' to '2020-06-11') ~ 15 days
# Soft Bounce (SB) Degrees of Freedom:

#233310:
df_exp_old_shared_CONTROL = old_shared_CONTROL_before_SB_datapoints + old_shared_CONTROL_after_SB_datapoints - 2
df_exp_old_shared_TEST    = old_shared_TEST_before_SB_datapoints + old_shared_TEST_after_SB_datapoints - 2

#bad_excellent:
df_exp_BE_CONTROL = BE_CONTROL_before_SB_datapoints + BE_CONTROL_after_SB_datapoints - 2
df_exp_BE_TEST    = BE_TEST_before_SB_datapoints + BE_TEST_after_SB_datapoints - 2

#Magic Johnson:
df_exp_MJ_CONTROL = MJ_CONTROL_before_SB_datapoints + MJ_CONTROL_after_SB_datapoints - 2
df_exp_MJ_TEST    = MJ_TEST_before_SB_datapoints + MJ_TEST_after_SB_datapoints - 2

In [ ]:
print('Degrees of Freedom:')
print(' ')
print('#233310:')
print('df_exp_old_shared_CONTROL = {}'.format(df_exp_old_shared_CONTROL))
print('df_exp_old_shared_TEST = {}'.format(df_exp_old_shared_TEST))
print(' ')
print('#bad_excellent:')
print('df_exp_BE_CONTROL = {}'.format(df_exp_BE_CONTROL))
print('df_exp_BE_TEST = {}'.format(df_exp_BE_TEST))
print(' ')
print('#Magic Johnson:')
print('df_exp_MJ_CONTROL = {}'.format(df_exp_MJ_CONTROL))
print('df_exp_MJ_TEST = {}'.format(df_exp_MJ_TEST))

Degrees of Freedom:
 
#233310:
df_exp_old_shared_CONTROL = 19
df_exp_old_shared_TEST = 16
 
#bad_excellent:
df_exp_BE_CONTROL = 19
df_exp_BE_TEST = 20
 
#Magic Johnson:
df_exp_MJ_CONTROL = 22
df_exp_MJ_TEST = 21


![t-test-chosen](https://lh4.googleusercontent.com/43wHV5uDMD4C6IVZgWpURqeBZ_j4YrVc3ooHlfV5EWp22c74Eb5ObH6HcFDVqKoo1c_6k2zqeeegcxqrhUok=w1920-h1007)

In [ ]:
# From the t-table:
#_______________________________________________________
# Alpha-value ("Significance Level") of 5%:   (Balanced Type I and Type II error probability)

#233310:
critical_value_exp_old_shared_CONTROL = 2.093  #19
critical_value_exp_old_shared_TEST = 2.120     #16

#bad_excellent:
critical_value_exp_BE_CONTROL = 2.093  #19
critical_value_exp_BE_TEST = 2.086  #20

#Magic Johnson:
critical_value_exp_MJ_CONTROL = 2.074  #22
critical_value_exp_MJ_TEST = 2.080  #21

# t-value - Unpaired Samples (Independent)

Unpaired Samples (Independent)

![t_value](https://lh4.googleusercontent.com/OcjIMZW9HaT0KOhfpVtSEepN6p57PGWp_12ZSlI489Z1LbF6Znj6EpEu0uMnQxRCbIVDE3OD-_EkT0u8uJvo=w1920-h1007-rw)

It is a Ratio between the difference **between** two groups (means) and the difference **within** the groups (variability)

*   A large t-value tells you that the groups **are different**.
*   A small t-value tells you that the groups **are similar**.

In [ ]:
# Unpaired Samples (Independent):

#233310:
t_value_exp_old_shared_CONTROL = (abs( (old_shared_CONTROL_before_SB_mean) - (old_shared_CONTROL_after_SB_mean) )) / (math.sqrt(( ( (old_shared_CONTROL_before_SB_variance) / (old_shared_CONTROL_before_SB_datapoints) ) + ( (old_shared_CONTROL_after_SB_variance) / (old_shared_CONTROL_after_SB_datapoints) ))))
t_value_exp_old_shared_TEST = (abs( (old_shared_TEST_before_SB_mean) - (old_shared_TEST_after_SB_mean) )) / (math.sqrt(( ( (old_shared_TEST_before_SB_variance) / (old_shared_TEST_before_SB_datapoints) ) + ( (old_shared_TEST_after_SB_variance) / (old_shared_TEST_after_SB_datapoints) ))))

#bad_excellent:
t_value_exp_BE_CONTROL = (abs( (BE_CONTROL_before_SB_mean) - (BE_CONTROL_after_SB_mean) )) / (math.sqrt(( ( (BE_CONTROL_before_SB_variance) / (BE_CONTROL_before_SB_datapoints) ) + ( (BE_CONTROL_after_SB_variance) / (BE_CONTROL_after_SB_datapoints) ))))
t_value_exp_BE_TEST = (abs( (BE_TEST_before_SB_mean) - (BE_TEST_after_SB_mean) )) / (math.sqrt(( ( (BE_TEST_before_SB_variance) / (BE_TEST_before_SB_datapoints) ) + ( (BE_TEST_after_SB_variance) / (BE_TEST_after_SB_datapoints) ))))

#Magic Johnson:
t_value_exp_MJ_CONTROL = (abs( (MJ_CONTROL_before_SB_mean) - (MJ_CONTROL_after_SB_mean) )) / (math.sqrt(( ( (MJ_CONTROL_before_SB_variance) / (MJ_CONTROL_before_SB_datapoints) ) + ( (MJ_CONTROL_after_SB_variance) / (MJ_CONTROL_after_SB_datapoints) ))))
t_value_exp_MJ_TEST = (abs( (MJ_TEST_before_SB_mean) - (MJ_TEST_after_SB_mean) )) / (math.sqrt(( ( (MJ_TEST_before_SB_variance) / (MJ_TEST_before_SB_datapoints) ) + ( (MJ_TEST_after_SB_variance) / (MJ_TEST_after_SB_datapoints) ))))

In [ ]:
print('T-Values:')
print(' ')
print('#233310:')
print('t_value_exp_old_shared_CONTROL = {}'.format(t_value_exp_old_shared_CONTROL))
print('t_value_exp_old_shared_TEST = {}'.format(t_value_exp_old_shared_TEST))
print(' ')
print('#bad_excellent:')
print('t_value_exp_BE_CONTROL = {}'.format(t_value_exp_BE_CONTROL))
print('t_value_exp_BE_TEST = {}'.format(t_value_exp_BE_TEST))
print(' ')
print('#Magic Johnson:')
print('t_value_exp_MJ_CONTROL = {}'.format(t_value_exp_MJ_CONTROL))
print('t_value_exp_MJ_TEST = {}'.format(t_value_exp_MJ_TEST))

T-Values:
 
#233310:
t_value_exp_old_shared_CONTROL = 0.08135611395154928
t_value_exp_old_shared_TEST = 0.5405841132154476
 
#bad_excellent:
t_value_exp_BE_CONTROL = 2.0149086557846996
t_value_exp_BE_TEST = 1.557890946465808
 
#Magic Johnson:
t_value_exp_MJ_CONTROL = 0.6997835809991174
t_value_exp_MJ_TEST = 0.8004381029457525


# T-Test Check - Unpaired Samples (Independent)

In [ ]:
# Verifying t-test:
# If T_Value > Critical_Value, then P_Value > .05

#233310:
print('#233310 - CONTROL:')
print(' ')
if t_value_exp_old_shared_CONTROL > critical_value_exp_old_shared_CONTROL:
    print("REJECT the H0 -> There is statistically significant difference between the samples")
else:
    print("Don't Reject the H0 -> There is NO statistically significant difference between the samples")
print('t_value_exp_old_shared_CONTROL = {}'.format(t_value_exp_old_shared_CONTROL))
print('critical_value_exp_old_shared_CONTROL = {}'.format(critical_value_exp_old_shared_CONTROL))
print(' ')
print(' ')
print('#233310 - TEST:')
print(' ')
if t_value_exp_old_shared_TEST > critical_value_exp_old_shared_TEST:
    print("REJECT the H0 -> There is statistically significant difference between the samples")
else:
    print("Don't Reject the H0 -> There is NO statistically significant difference between the samples")
print('t_value_exp_old_shared_TEST = {}'.format(t_value_exp_old_shared_TEST))
print('critical_value_exp_old_shared_TEST = {}'.format(critical_value_exp_old_shared_TEST))
print('________________________________________________________________________________________________________')
print(' ')

#______________________________________________________________________________________________________________
#bad_excellent:
print('#bad_excellent - CONTROL:')
print(' ')
if t_value_exp_BE_CONTROL > critical_value_exp_BE_CONTROL:
    print("REJECT the H0 -> There is statistically significant difference between the samples")
else:
    print("Don't Reject the H0 -> There is NO statistically significant difference between the samples")
print('t_value_exp_BE_CONTROL = {}'.format(t_value_exp_BE_CONTROL))
print('critical_value_exp_BE_CONTROL = {}'.format(critical_value_exp_BE_CONTROL))
print(' ')
print(' ')
print('#bad_excellent - TEST:')
print(' ')
if t_value_exp_BE_TEST > critical_value_exp_BE_TEST:
    print("REJECT the H0 -> There is statistically significant difference between the samples")
else:
    print("Don't Reject the H0 -> There is NO statistically significant difference between the samples")
print('t_value_exp_BE_TEST = {}'.format(t_value_exp_BE_TEST))
print('critical_value_exp_BE_TEST = {}'.format(critical_value_exp_BE_TEST))
print('________________________________________________________________________________________________________')
print(' ')

#______________________________________________________________________________________________________________
#Magic Johnson:
print('#Magic Johnson - CONTROL:')
print(' ')
if t_value_exp_MJ_CONTROL > critical_value_exp_MJ_CONTROL:
    print("REJECT the H0 -> There is statistically significant difference between the samples")
else:
    print("Don't Reject the H0 -> There is NO statistically significant difference between the samples")
print('t_value_exp_MJ_CONTROL = {}'.format(t_value_exp_MJ_CONTROL))
print('critical_value_exp_MJ_CONTROL = {}'.format(critical_value_exp_MJ_CONTROL))
print(' ')
print(' ')
print('#Magic Johnson - TEST:')
print(' ')
if t_value_exp_MJ_TEST > critical_value_exp_MJ_TEST:
    print("REJECT the H0 -> There is statistically significant difference between the samples")
else:
    print("Don't Reject the H0 -> There is NO statistically significant difference between the samples")
print('t_value_exp_MJ_TEST = {}'.format(t_value_exp_MJ_TEST))
print('critical_value_exp_MJ_TEST = {}'.format(critical_value_exp_MJ_TEST))

#233310 - CONTROL:
 
Don't Reject the H0 -> There is NO statistically significant difference between the samples
t_value_exp_old_shared_CONTROL = 0.08135611395154928
critical_value_exp_old_shared_CONTROL = 2.093
 
 
#233310 - TEST:
 
Don't Reject the H0 -> There is NO statistically significant difference between the samples
t_value_exp_old_shared_TEST = 0.5405841132154476
critical_value_exp_old_shared_TEST = 2.12
________________________________________________________________________________________________________
 
#bad_excellent - CONTROL:
 
Don't Reject the H0 -> There is NO statistically significant difference between the samples
t_value_exp_BE_CONTROL = 2.0149086557846996
critical_value_exp_BE_CONTROL = 2.093
 
 
#bad_excellent - TEST:
 
Don't Reject the H0 -> There is NO statistically significant difference between the samples
t_value_exp_BE_TEST = 1.557890946465808
critical_value_exp_BE_TEST = 2.086
______________________________________________________________________________